In [1]:
import pandas as pd
import csv
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import re
import jieba
# gensim用来加預預訓練word vector
from gensim.models import KeyedVectors
import warnings
warnings.filterwarnings("ignore")
from tensorflow import keras
from tensorflow.python.keras.models import Sequential
# from tensorflow.python.keras.layers import Dense, GRU, Embedding, LSTM, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import  Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [2]:
import demoji

def emojiRemove(text):
    result = demoji.replace(text, "")
    return result
emojiRemove("燕麥凍有點失望，為什麼是圖中這樣碎碎的，喝起來像在喝渣渣，燕麥凍不應該是一整塊大塊的嗎😭😭而且燕麥凍要加15元😭")

'燕麥凍有點失望，為什麼是圖中這樣碎碎的，喝起來像在喝渣渣，燕麥凍不應該是一整塊大塊的嗎而且燕麥凍要加15元'

In [272]:
# 將正負面詞語彙總至一個list
train_texts_orig = []

# 正負面句子數量要相同
posi_num = 320
nega_num = posi_num

with open("positive2.txt", encoding = "utf-8") as f:
    line = list(csv.reader(f, delimiter=','))
    for i in range(posi_num):
        train_texts_orig.append(line[i][0])

with open("negative2.txt", encoding = "utf-8") as f:
    line = list(csv.reader(f, delimiter=','))
    for i in range(nega_num):
        train_texts_orig.append(line[i][0])

print("train_texts_orig內容數量:",len(train_texts_orig))
print(train_texts_orig)

train_texts_orig內容數量: 640
['剛試喝一口的奶蓋味道 還是手中的飲料 傳來陣陣撲鼻的香氣 就感覺自己整路都被甜點包圍著', '飲料很好喝 金萱/奶茶/花果茶... 都很不錯', '聽說奶蓋很厲害😆 試了金萱烏龍➕奶蓋 ', '奶蓋的份量我覺得給的很足夠 第一口喝的時候茶只能喝到一點點就知道奶蓋有厚度', '有吃起司蛋糕的那種口感搭配金萱烏龍很配 茶也很好喝，很香', '喜歡他們家的玉露抹茶 蠻多人也推四季春烏龍茶、珍珠奶茶、檸檬優酪 他們家的檸檬是原汁、很酸很過癮😆', '喜歡這家的茶拿鐵，茶味夠牛奶也可以隨客人喜好選擇！', '平價好喝！可以選鮮奶品牌滿特別的！小芋圓也好吃！', '珍珠Q軟超級好吃，是這附近煮的最好吃的珍珠！', '飲料品項很多  原茶茶味濃郁 不會苦澀', '茶味夠，非常喜歡，特別喜歡他們到桂花烏龍', '很有特色的飲料店，點了桂花釀，口味不錯，還會再來買！', '這附近最好喝的手搖飲料！', '喝了這麼多次，品質始終保持穩定，相當難得', '真實茶葉看的見，用咖啡機現泡現沖！', '東西好喝，女店員服務態度很好', '豪喝 第一口被驚艷到 點了熱門的桂花烏龍拿鐵微糖微冰 比comeback的桂花好喝哈哈桂花香氣很夠 天然的那種 不是桂花糖漿 店前的貓貓很可愛-.-', '桂花烏龍拿鐵好喝！3種牛奶可選真的超棒！', '推玄米茶拿鐵與桂花烏龍拿鐵，好喝', '被埋沒的神店 茶類飲品都超讚 加牛奶又超對味', '飲品製作快速也很好喝', '拿鐵好喝，鮮奶也是市面上看得到的品牌', '飲料好喝，有特殊茶香', '台北最好喝茶！鮮奶茶超棒！大推炭培和玄米', '茶是用煮茶機現煮的，好喝但價錢又便宜，CP很高', '豪好喝！值得細細品味的好茶店！', '茶類很推薦，好喝的一間店', '桂花烏龍奶茶好喝~', 'Super好喝 大家不准跟我搶', '好喝實在，CP值高', '推鐵觀音奶茶', '店員好親切 謝謝你們❤❤ 玉荷冰綠加荔枝凍好喝～微糖滿剛好的！', '店員親切有禮貌做飲料又快！', '芭樂檸檬綠最適合夏日來一杯 店面乾淨、服務態度不錯', '製餐快速、店員也很有禮貌', '優質店家，飲料好喝 。', '店面裝潢.設計，還有店員服務.態度，在飲料店裡算是很不錯的', '之前有抽到免單過！很棒的體驗！', '飲料選擇多，室

In [3]:
# 使用gensim載入祠向量模型
cn_model_sg = KeyedVectors.load_word2vec_format('skip_gram_all_vector.txt', unicode_errors="ignore")

In [274]:
# 進行分詞和tokenize
# train_tokens是一個長長的list，其中含有4000个小list，對應每一則評論
train_tokens = []
for text in train_texts_orig:
    # print(text)
    # 去除標點符號
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    text = emojiRemove(text)
    print(text)
    # 建立jieba生成器
    cut = jieba.cut(text)
    # print(cut)
    # 把生成器轉換為list
    cut_list = [ i for i in cut ]
    print(cut_list)
    for i, word in enumerate(cut_list):
        try:
            # 將辭轉換為索引index
            cut_list[i] = cn_model_sg.key_to_index[word]
        except KeyError:
            # 如果詞不在字典中，輸出0
            cut_list[i] = 0
    # print(cut_list)
    train_tokens.append(cut_list)

剛試喝一口的奶蓋味道還是手中的飲料傳來陣陣撲鼻的香氣就感覺自己整路都被甜點包圍著
['剛試', '喝', '一口', '的', '奶蓋', '味道', '還是', '手中', '的', '飲料', '傳來', '陣陣', '撲', '鼻', '的', '香氣', '就', '感覺', '自己', '整路', '都', '被', '甜點', '包圍', '著']
飲料很好喝金萱奶茶花果茶都很不錯
['飲料', '很', '好喝', '金萱', '奶茶', '花果茶', '都', '很', '不錯']
聽說奶蓋很厲害試了金萱烏龍奶蓋
['聽', '說', '奶蓋', '很', '厲害', '試', '了', '金萱', '烏龍', '奶蓋']
奶蓋的份量我覺得給的很足夠第一口喝的時候茶只能喝到一點點就知道奶蓋有厚度
['奶蓋', '的', '份量', '我覺', '得給', '的', '很', '足夠', '第一口', '喝', '的', '時候', '茶', '只能', '喝到', '一點點', '就', '知道', '奶蓋', '有', '厚度']
有吃起司蛋糕的那種口感搭配金萱烏龍很配茶也很好喝很香
['有', '吃', '起司', '蛋糕', '的', '那種', '口感', '搭配', '金萱', '烏龍', '很配', '茶', '也', '很', '好喝', '很香']
喜歡他們家的玉露抹茶蠻多人也推四季春烏龍茶珍珠奶茶檸檬優酪他們家的檸檬是原汁很酸很過癮
['喜歡', '他們', '家', '的', '玉露', '抹', '茶', '蠻', '多', '人', '也', '推', '四季春', '烏龍', '茶', '珍珠奶茶', '檸檬', '優酪', '他們', '家', '的', '檸檬是', '原汁', '很酸', '很', '過癮']
喜歡這家的茶拿鐵茶味夠牛奶也可以隨客人喜好選擇
['喜歡', '這家', '的', '茶', '拿', '鐵', '茶味夠', '牛奶', '也', '可以', '隨', '客人', '喜好', '選擇']
平價好喝可以選鮮奶品牌滿特別的小芋圓也好吃
['平價', '好喝', '可以', '選鮮奶', '品牌', '滿特別', '的', '小芋', '圓也', '好吃']
珍珠Q軟超級好吃是這附近煮的最好吃的珍珠

In [275]:
print(train_tokens)

[[377972, 908, 2995, 0, 113061, 1529, 91, 2948, 0, 3141, 8491, 16139, 4841, 6499, 0, 8336, 11, 289, 33, 280899, 13, 40, 16468, 8985, 4792], [3141, 37, 15282, 501745, 8424, 252050, 13, 37, 672], [302, 17, 113061, 37, 2355, 2617, 2, 501745, 14991, 113061], [113061, 0, 22701, 2288, 34454, 0, 37, 1572, 58921, 908, 0, 75, 1517, 764, 38047, 6171, 11, 106, 113061, 6, 6315], [6, 593, 63029, 4129, 0, 2358, 3808, 986, 501745, 14991, 116555, 1517, 8, 37, 15282, 76256], [187, 69, 265, 0, 75743, 3963, 1517, 4383, 44, 15, 8, 2923, 224443, 14991, 1517, 94099, 9826, 94365, 69, 265, 0, 153206, 43075, 293324, 37, 15436], [187, 3865, 0, 1517, 549, 2158, 0, 2401, 8, 25, 2156, 3459, 5760, 157], [16555, 15282, 25, 0, 294, 2212538, 0, 807178, 988114, 6192], [7568, 4238, 0, 6192, 1, 26, 729, 1871, 0, 64, 6192, 0, 7568], [3141, 190060, 88, 1869, 71601, 4539, 6699, 195, 23723], [0, 141, 187, 276, 187, 69, 28, 10087, 14991], [37, 6, 995, 0, 3141, 286472, 2, 10087, 18794, 3237, 672, 1633, 104, 22, 181], [26, 729,

In [276]:
# num_tokens統計個句子的長度
num_tokens = [ len(tokens) for tokens in train_tokens ]
num_tokens = np.array(num_tokens)
np.mean(num_tokens)
np.max(num_tokens)

161

In [277]:
# 以平均加上兩倍標準差作為訓練長度(RNN需統一訓練字句的長度)
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

53

In [278]:
# 篩選出可用的句子數量(長度 < mean + 2*std)
np.sum( num_tokens < max_tokens ) / len(num_tokens)

0.953125

In [279]:
# 把索引值變回文字(依照特定長度)
def reverse_tokens(tokens):
    text = ''
    for i in tokens:
        if i != 0:
            text = text + cn_model_sg.index_to_key[i] + " "
        else:
            text = text + ' '
    return text

In [280]:
reverse = reverse_tokens(train_tokens[0])
print(train_texts_orig[0])
reverse

剛試喝一口的奶蓋味道 還是手中的飲料 傳來陣陣撲鼻的香氣 就感覺自己整路都被甜點包圍著


'剛試 喝 一口  奶蓋 味道 還是 手中  飲料 傳來 陣陣 撲 鼻  香氣 就 感覺 自己 整路 都 被 甜點 包圍 著 '

In [281]:
# 遷入層維度
embedding_dim = 250

In [282]:
# 存取索引值0~50000的祠向量
# 只使用前50000個詞
num_words = 3096281
# 初始化embedding_matrix，之後在keras上進行應用(??)
embedding_matrix = np.zeros((num_words, embedding_dim))
# embedding_matrix為一個 [num_words，embedding_dim] 的矩陣
# 维度为 50000 * 250
for i in range(num_words):
    embedding_matrix[i,:] = cn_model_sg[cn_model_sg.index_to_key[i]]
embedding_matrix = embedding_matrix.astype('float32')
print(embedding_matrix)

[[ 0.24608016 -0.07148311  0.20930456 ...  0.22566666 -0.17532614
   0.09889419]
 [ 0.4072783  -0.03727184  0.30424917 ...  0.3245045  -0.27184546
  -0.20718965]
 [ 0.26537406 -0.35084382  0.22699125 ...  0.21376689 -0.23796429
   0.09573323]
 ...
 [ 0.06433456  0.10668301  0.09491172 ... -0.05254407  0.00441413
   0.10814877]
 [-0.02341211  0.06919388  0.02343448 ... -0.01754162  0.11114478
   0.08314369]
 [ 0.05430191 -0.02122357  0.15093148 ...  0.02288825 -0.01672576
   0.06525383]]


In [283]:
# 確認有一個字的向量是幾維
np.sum( cn_model_sg[cn_model_sg.index_to_key[333]] == embedding_matrix[333] )

250

In [284]:
embedding_matrix.shape

(3096281, 250)

In [285]:
# 將句子截斷或補成相同長度
#                           sequences, 序列的最大長度
train_pad = pad_sequences(train_tokens, maxlen=max_tokens,
#                        若需要補0，補在前面, 若需要切，從頭切
                            padding='pre', truncating='pre')
train_pad.shape

(640, 53)

In [286]:
# i forgor why QQ
train_pad[ train_pad >= num_words ] = 0

In [287]:
train_pad[33]

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0, 137775, 622547,     64,    708,  10001,     22,
         5214,   8038,   2174,  31578,    672])

In [288]:
# 標註label，正向為1，負向為0
train_target = np.concatenate( (np.ones(posi_num),np.zeros(nega_num)) )
train_target

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [289]:
from sklearn.model_selection import train_test_split

In [290]:
# 特徵為train_pad(經截長補短、索引化)，標籤為0/1，切分為90%訓練集，10%測試集
X_train, X_test, y_train, y_test = train_test_split(train_pad,
                                                    train_target,
                                                    test_size=0.1,
                                                    random_state=12)

In [291]:
test = np.random.randint(180)
print(test)
print(reverse_tokens(X_train[test]))
print('class: ',y_train[test])

77
                        漲到 一百 了 其實 漲價 沒關 係 好像 肉 變少 了 以前 都 會 滿出 來現 在 整個 像 小碗  只有 三塊 肉 吧 可能 不再 關顧 了 
class:  0.0


In [295]:
# model = keras.models.load_model("model/98.4.3")

In [296]:
model = Sequential()

In [297]:
model.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_tokens,
                    trainable=False))

In [298]:
model.add(Bidirectional(LSTM(units=32, return_sequences=True)))
model.add(LSTM(units=16, return_sequences=False))

In [299]:
# model.add(Dense(1))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

# 我们使用adam以0.001的learning rate进行优化
optimizer = Adam(lr=1e-3)

In [300]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])

In [301]:
# model.summary()

In [302]:
# model.fit(X_train, y_train)

In [303]:
# from keras.utils import plot_model

# plot_model(model,to_file='model.png')

In [304]:
path_checkpoint = 'sentiment_checkpoint.keras'
checkpoint = ModelCheckpoint(filepath=path_checkpoint, monitor='val_loss',
                                      verbose=1, save_weights_only=True,
                                      save_best_only=True)

In [305]:
try:
    model.load_weights(path_checkpoint)
except Exception as e:
    print(e)

Unable to load weights saved in HDF5 format into a subclassed Model which has not created its variables yet. Call the Model first, then load the weights.


In [306]:
earlystopping = EarlyStopping(monitor='val_loss', patience=4, verbose=1)

In [307]:
lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1, min_lr=1e-5, patience=0,
                                       verbose=1)

In [308]:
callbacks = [
    earlystopping, 
    checkpoint,
    lr_reduction
]

In [309]:
model.fit(X_train, y_train,
          validation_split=0.1, 
          epochs=20,
          batch_size=20,
          callbacks=callbacks)

Epoch 1/20
26/26 [==============================] - 16s 110ms/step - loss: 0.6386 - accuracy: 0.6313 - val_loss: 0.5821 - val_accuracy: 0.6897

Epoch 1: val_loss improved from inf to 0.58215, saving model to sentiment_checkpoint.keras
Epoch 2/20
26/26 [==============================] - 2s 64ms/step - loss: 0.4531 - accuracy: 0.8166 - val_loss: 0.4633 - val_accuracy: 0.7414

Epoch 2: val_loss improved from 0.58215 to 0.46325, saving model to sentiment_checkpoint.keras
Epoch 3/20
26/26 [==============================] - 2s 65ms/step - loss: 0.3497 - accuracy: 0.8629 - val_loss: 0.4355 - val_accuracy: 0.7931

Epoch 3: val_loss improved from 0.46325 to 0.43555, saving model to sentiment_checkpoint.keras
Epoch 4/20
26/26 [==============================] - 2s 64ms/step - loss: 0.2895 - accuracy: 0.8803 - val_loss: 0.3520 - val_accuracy: 0.8448

Epoch 4: val_loss improved from 0.43555 to 0.35200, saving model to sentiment_checkpoint.keras
Epoch 5/20
26/26 [==============================] - 2s

In [310]:
result = model.evaluate(X_test, y_test)
print('Accuracy:{0:.2%}'.format(result[1]))

2/2 [==============================] - 0s 47ms/step - loss: 0.2699 - accuracy: 0.9062
Accuracy:90.62%


In [311]:
def predict_sentiment(text):
    # print(text)
    # 去除編點符號
    text = re.sub("[\s+\.\!\/_,$%^*(+\"\']+|[+——！，。？、~@#￥%……&*（）]+", "",text)
    text = emojiRemove(text)
    print(text)
    # 段詞器
    cut = jieba.cut(text)
    cut_list = [ i for i in cut ]
    # tokenize
    for i, word in enumerate(cut_list):
        try:
            cut_list[i] = cn_model_sg.key_to_index[word]
        except KeyError:
            cut_list[i] = 0
    # padding
    tokens_pad = pad_sequences([cut_list], maxlen=max_tokens,
                           padding='pre', truncating='pre')
    # print(tokens_pad)
    # 預測
    result = model.predict(tokens_pad)
    coef = result[0][0]
    if coef >= 0.5:
        print('V 正面評價','output=%.2f'%coef)
    else:
        print('X 負面評價','output=%.2f'%coef)

In [312]:
test_list = [
    "皮很脆，內餡吃得到魷魚塊，即使在人煙稀少的巷弄裡，生意還是很好，證明食物本身夠強，很難得在草漯能吃到比市區好吃的東西，珍惜啊!", 
    "每顆吃到的時候都還是脆的，裡面的章魚肉也都很大塊，上面的配料8顆吃完都還有剩，超讚的 還有可愛的狗狗在裡面", 
    "店內空間大，座位多，餐點好吃，泡菜意麵不會很辣，泡菜吃起來酸甜解膩，紅燒牛肉麵加上酸菜也很好吃，會再次回訪的店家", 
    "雖然和旗艦店沒辦法比，但和差不多價位的鍋店比，也算是可以的了，希望貴店還有進步空間！另外覺得在草漯能有宵夜火鍋能吃太幸福了！", 
    "蔥油餅必吃，可先吃原味，趁熱吃可以吃到麵皮的香味，再沾辣椒醬吃，美味！牛排好吃，濃湯很水很稀。",
    "好喝無糖也不會很奇怪愛喝奶蓋的朋友別錯過奶蓋杯超級貼心設計服務員態度很好飲料製作也很有效率很方便有插座可以充電之前有抽到免單過！很棒的體驗！"
]
for text in test_list:

    predict_sentiment(text)


皮很脆內餡吃得到魷魚塊即使在人煙稀少的巷弄裡生意還是很好證明食物本身夠強很難得在草漯能吃到比市區好吃的東西珍惜啊
V 正面評價 output=0.94
每顆吃到的時候都還是脆的裡面的章魚肉也都很大塊上面的配料8顆吃完都還有剩超讚的還有可愛的狗狗在裡面
V 正面評價 output=0.82
店內空間大座位多餐點好吃泡菜意麵不會很辣泡菜吃起來酸甜解膩紅燒牛肉麵加上酸菜也很好吃會再次回訪的店家
V 正面評價 output=0.98
雖然和旗艦店沒辦法比但和差不多價位的鍋店比也算是可以的了希望貴店還有進步空間另外覺得在草漯能有宵夜火鍋能吃太幸福了
X 負面評價 output=0.36
蔥油餅必吃可先吃原味趁熱吃可以吃到麵皮的香味再沾辣椒醬吃美味牛排好吃濃湯很水很稀
V 正面評價 output=0.93
好喝無糖也不會很奇怪愛喝奶蓋的朋友別錯過奶蓋杯超級貼心設計服務員態度很好飲料製作也很有效率很方便有插座可以充電之前有抽到免單過很棒的體驗
V 正面評價 output=0.96


In [315]:
test_list = [
    "老闆明顯安排人力不足，請務必要改善，補足人力或不應接這麼多單", 
    "傻眼，買皮蛋瘦肉粥...看老闆一個皮蛋煮兩碗，一顆小雞蛋煮兩碗粥。雖然我是不知道其他人是不是這樣賣，但看到他裝在一個平常我們去紅豆之家買大碗燒仙草的那種碗裡，然後賣你７５元，以前只是覺得小貴，現在是整個覺得好貴",
     "我點咖哩香菇雞絲麵，結果香菇又小又少，還以為是泡麵裡調味包的香菇，價錢漲這麼多料卻變少這麼多有點失望，之前來吃還沒這麼誇張。", 
     "辣椒很香但牛肉麵都肉太硬、乾米苔目味道太淡不好吃", 
    "結帳的沒什麼服務態度，牛肉麵清淡、麵條很爛沒嚼勁，量不多水餃很空虛小菜很清淡。",
    "點餐人員口氣沒有很好 名牌掛實習店長然後這種服務態度？然後有一名男店員 手有刺青的 客人不清楚餐點沒有向客人解釋就算了 半路跑出來口氣很差的說這兩種餐點是不一樣的 請問一下整間店的服務人員都這樣嗎？"
]
for text in test_list:

    predict_sentiment(text)


老闆明顯安排人力不足請務必要改善補足人力或不應接這麼多單
X 負面評價 output=0.05
傻眼買皮蛋瘦肉粥看老闆一個皮蛋煮兩碗一顆小雞蛋煮兩碗粥雖然我是不知道其他人是不是這樣賣但看到他裝在一個平常我們去紅豆之家買大碗燒仙草的那種碗裡然後賣你７５元以前只是覺得小貴現在是整個覺得好貴
X 負面評價 output=0.04
我點咖哩香菇雞絲麵結果香菇又小又少還以為是泡麵裡調味包的香菇價錢漲這麼多料卻變少這麼多有點失望之前來吃還沒這麼誇張
X 負面評價 output=0.15
辣椒很香但牛肉麵都肉太硬乾米苔目味道太淡不好吃
X 負面評價 output=0.36
結帳的沒什麼服務態度牛肉麵清淡麵條很爛沒嚼勁量不多水餃很空虛小菜很清淡
V 正面評價 output=0.97
點餐人員口氣沒有很好名牌掛實習店長然後這種服務態度然後有一名男店員手有刺青的客人不清楚餐點沒有向客人解釋就算了半路跑出來口氣很差的說這兩種餐點是不一樣的請問一下整間店的服務人員都這樣嗎
X 負面評價 output=0.05
